In [1]:
!pip install tensorflow==1.15
!pip install stable-baselines[mpi]==2.10.2

In [2]:
import os
import gym
import numpy as np
import pandas as pd
from gym import spaces

/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [ ]:
class LogEnvironment(gym.Env):
    metadata = {'render.modes': ['console']}
    
    num_envs = 1
    NoLog = 0
    Log = 1
    predicts=[]
    
    def __init__(self,dataset, high = None, low = None):
        super(LogEnvironment, self).__init__()
        self.rewards = []
        self.dataset = dataset
        self.states = np.array(dataset.iloc[:,:-1])
        self.state = 0
        self.predict_classes = []
        
        n_actions = 2
        self.action_space = spaces.Discrete(n_actions)
        if high is None:
            low = np.array(dataset.iloc[:,:-1].min()).astype(np.float32)
            high = np.array(dataset.iloc[:,:-1].max()).astype(np.float32)
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        else:
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        
    
    def reset(self):
        self.state = 0
        LogEnvironment.predicts.append(self.predict_classes)
        self.predict_classes = []
        return self.states[self.state].astype(np.float32)
    
    def step(self, action):
        if action == self.NoLog:
            self.predict_classes.append(0)
        elif action == self.Log:
            self.predict_classes.append(1)
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))
        
        done = bool(self.state >= len(self.classes)-1)
        
        
        
        reward = 1 if self.predict_classes[self.state] == self.classes[self.state] else 0
        self.rewards.append(reward)
        
        self.state += 0 if done else 1
        
        info = {}
        
        return self.states[self.state].astype(np.float32), reward, done, info
        